In [1]:
# minesweeper

In [2]:
import numpy as np

In [3]:
mine_grid: boolean 2D array (True = mine, False = empty)
count_grid: integer 2D array (0-8, neighbor mine counts)
display_grid: state 2D array (HIDDEN/REVEALED/FLAGGED)

# Display states
HIDDEN = 0
REVEALED = 1  
FLAGGED = 2

# Game states
PLAYING = 0
WON = 1
LOST = 2

# Display symbols
SYMBOLS = {
    HIDDEN: '□',      # Or '.' or '#'
    FLAGGED: 'F',     # Or '⚑'
    'MINE': '*',      # Or '💣'
    'EMPTY': ' '      # Revealed empty cell
    # Numbers 1-8 display as themselves
}

SyntaxError: invalid decimal literal (446482966.py, line 1)

In [ ]:
# Generated board (10x8)
[["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","#","#","#","#","#","#","#"]]

# With some revealed examples
[["#","#","#","#","#","#","#","#","#","#"]
 ["#","#","#","F","F",".","#","#","#","#"]
 ["#","#","#","F","3",".","#","#","#","#"]
 ["#","#","#",".",".",".",".","#","#","#"]
 ["#","#",".",".",".","1","F","#","#","#"]
 [".",".",".","2",".",".",".","#","#","#"]
 ["1","2","F","F",".","#","#","#","#","#"]
 [".","F",".","#","#","#","#","#","#","#"]]

# KEY:
# "#" unrevealed
# "F" flagged mine
# "." revealed
# "1-8" number of surrounding mines

In [5]:
import random

In [46]:
# function to generate board:
def generate_board(n_row, n_col, num_mines):
    # create lists with given dimensions + add random mines
    board = []
    for _ in range(n_row):
        rows = []
        for _ in range(n_col):
            rows.append(0)
        board.append(rows)
    mines = set()
    while len(mines) < num_mines:
        r = random.randint(0, n_row - 1)
        c = random.randint(0, n_col - 1)
        if (r, c) not in mines:
            board[r][c] = 'X'
            mines.add((r,c))
    # add numbers of mines
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for r in range(n_row):
        for c in range(n_col):
            if board[r][c] == 'X':
                continue
            count = 0
            for dr, dc in directions:
                nr = r + dr
                nc = c + dc
                if 0 <= nr < n_row and 0 <= nc < n_col:
                    if board[nr][nc] == 'X':
                        count += 1
            board[r][c] = count
    return board, mines

def type_coords(board, thing):
    coords = set()
    for r, row in enumerate(board):
        for c, val in enumerate(row):
            if val == thing:
                coords.add((r, c))
    return coords

# prints board with correct dimensions and unrevealed cells
def display_board(board): 
    covered_board = []
    for i in range(len(board)):
        covered_rows = []
        for i in range(len(board[0])):
            covered_rows.append("#")
        covered_board.append(covered_rows)
    return covered_board

# reveal cell (like left click)
def reveal_cell(board, covered_board, r, c):
    if covered_board[r][c] != "#":
        return True  # already revealed
    if board[r][c] == "X":
        covered_board[r][c] = "X"
        return False  # game over

    # Flood fill if it's 0
    to_reveal = [(r, c)]
    directions = [(-1, -1), (-1, 0), (-1, 1),(0, -1),(0, 1),(1, -1), (1, 0),(1, 1)]
    while to_reveal:
        x, y = to_reveal.pop()
        if covered_board[x][y] != "#":
            continue
        covered_board[x][y] = board[x][y]
        if board[x][y] == 0:
            for dr, dc in directions:
                nx, ny = x + dr, y + dc
                if 0 <= nx < len(board) and 0 <= ny < len(board[0]):
                    if covered_board[nx][ny] == "#":
                        to_reveal.append((nx, ny))
    return True

# flag cell (like right click)
def flag_cell(covered_board,r,c):
    if covered_board[r][c] == "F":
        covered_board[r][c] = "#"
    elif covered_board[r][c] == "#":
        covered_board[r][c] = "F"
    return covered_board

def check_win(board, covered_board, mine_coords): # check coords of flags = mines, no more hash on board
    flagged = set()
    unrevealed = 0

    for r in range(len(covered_board)):
        for c in range(len(covered_board[0])):
            cell = covered_board[r][c]
            if cell == '#':
                unrevealed += 1
            elif cell == 'F':
                flagged.add((r, c))

    # Check: all mines flagged, and no hidden cells left
    return flagged == mine_coords and unrevealed == 0

def reveal_all_mines(board, covered_board):
    for r in range(len(board)):
        for c in range(len(board[0])):
            if board[r][c] == 'X':
                covered_board[r][c] = 'X'
    return covered_board

def prettyprint(board):
    as_str = []
    for row in board:
        as_str.append(str(row))
    return ("\n".join(as_str))


def play_game(rows=8, cols=10, num_mines=10):
    board, mine_coords = generate_board(rows,cols,num_mines)
    covered = display_board(board)

    while True:
        print("+------------------------------------------------+")
        print(prettyprint(covered))
        print("+------------------------------------------------+")
        action = input("Enter move (R/F row col):").split() # get player's choice

        # check if there are 3 values in input
        if len(action) != 3:
            print("❌ Invalid input, use this format: f 1 2 or r 0 0!")
            continue
        # check if row and col are numbers
        action, r_str, c_str = action
        if not (r_str.isdigit() and c_str.isdigit()):
            print("❌ Row and column must be numbers!")
            continue
        # check if coord is within board    
        r = int(r_str)
        c = int(c_str)
        if not (0 <= r < rows and 0 <= c < cols):
            print("❌ Coordinates out of bounds.")
            continue

        # if player reveal or flag
        if action == "R":
            alive = reveal_cell(board, covered, r, c)
            if not alive:
                reveal_all_mines(board, covered)
                print(prettyprint(covered))
                print("💥 You hit a mine... GAME OVER!")
                break
        elif action == "F":
            flag_cell(covered, r, c)
        else:
            print("❌ Unknown action! Use 'R' to reveal or 'F' to flag.")
            continue

        if check_win(covered, covered_board, mine_coords):
            prettyprint(covered)
            print("You won! Congratulations.")
            break

#if __name__ == "__main__":
#    play_game()

In [ ]:
type_coords(board,"X")

In [11]:
board,mines = generate_board(8,10,8)
print(prettyprint(board))
print(f"Mine coords: {mines}")

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 'X', 2]
[0, 0, 0, 0, 0, 0, 1, 2, 3, 'X']
[0, 0, 0, 0, 0, 0, 1, 'X', 2, 1]
[0, 0, 1, 2, 2, 2, 2, 2, 1, 0]
[0, 0, 1, 'X', 'X', 2, 'X', 2, 1, 0]
[1, 1, 2, 2, 2, 2, 2, 'X', 1, 0]
[1, 'X', 1, 0, 0, 0, 1, 1, 1, 0]
Mine coords: {(7, 1), (3, 7), (5, 4), (1, 8), (6, 7), (2, 9), (5, 6), (5, 3)}


In [12]:
covered_board = (display_board(board))
covered_board

[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [13]:
board

[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 'X', 2],
 [0, 0, 0, 0, 0, 0, 1, 2, 3, 'X'],
 [0, 0, 0, 0, 0, 0, 1, 'X', 2, 1],
 [0, 0, 1, 2, 2, 2, 2, 2, 1, 0],
 [0, 0, 1, 'X', 'X', 2, 'X', 2, 1, 0],
 [1, 1, 2, 2, 2, 2, 2, 'X', 1, 0],
 [1, 'X', 1, 0, 0, 0, 1, 1, 1, 0]]

In [14]:
flag_cell(board,covered_board,0,2)

[['#', '#', 'F', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [15]:
reveal_cell(board,covered_board,0,3)

True

In [47]:
play_game()

+------------------------------------------------+
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
+------------------------------------------------+


Enter move (R/F row col): R 0 0


+------------------------------------------------+
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[1, 2, '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
+------------------------------------------------+


Enter move (R/F row col): R 3 0


[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, 'X', '#', 'X', '#', '#', 'X', '#', '#']
[1, 2, '#', 'X', '#', '#', '#', '#', '#', '#']
['X', '#', '#', 'X', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', 'X', '#']
['#', '#', '#', '#', '#', '#', '#', '#', 'X', '#']
['#', '#', 'X', '#', '#', '#', 'X', '#', '#', '#']
💥 You hit a mine... GAME OVER!


In [ ]:
play_game()

+------------------------------------------------+
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
+------------------------------------------------+


Enter move (R/F row col): r 0 0


❌ Unknown action! Use 'R' to reveal or 'F' to flag.
+------------------------------------------------+
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
+------------------------------------------------+


Enter move (R/F row col): R 0 0


+------------------------------------------------+
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, '#', '#', '#', '#', '#', '#', '#', '#']
[0, 1, 2, 3, 2, 1, '#', '#', '#', '#']
[0, 0, 0, 0, 0, 1, '#', '#', '#', '#']
[0, 0, 0, 0, 0, 1, '#', '#', '#', '#']
+------------------------------------------------+


In [ ]:
0 0 0 0 0 1 # # # #
0 0 0 0 0 1 # # # #
0 1 1 1 0 1 # # # #
0 1 F 1 0 1 # # # #
0 1 1 1 1 2 # # # #
0 0 0 0 1 F 2 # # #
0 0 0 0 1 1 2 # # #
0 0 0 0 0 0 1 # # #